In [1]:
l = [ "People wearing harnesses using ropes to climb up a rock slope.", "A person climbing down a rock edge while someone talks about donating to a cause.", "A person is repelling down a rock while a girl asks for donations.", "A woman is describing some fears you may have while rock climbing.", "A woman is shown rock climbing and coming down from the rock, she then hugs a man when she is down.", "a man going down a rock cliff using a harness and rope with another person nearby", "A woman is narrating in the background as mountain climbers scale the side of a mountain.", "A woman is talking as people are using ropes lo lower themselves down a mountain.", "A group of people watch as woman scales down the side of a mountain.", "A woman climbs on a rock formation using safety ropes to protect her." ]

print(len(l))

10


In [2]:
import json
import os
from pathlib import Path

from tqdm import tqdm

TRANSLATED_METADATA_PATH = f"{Path.home()}/data/vast-2m/metadata"
ORIGINAL_METADATA_PATH = f"{Path.home()}/data/vast-2m/vast_2m_chunk_e"
TEMP_REFINED_METADATA_PATH = f"{Path.home()}/data/vast-2m/metadata-refined-temp"

os.makedirs(TEMP_REFINED_METADATA_PATH, exist_ok=True)

for i in tqdm(range(500)):
  if i == 301:
    original_metadata_file = f"{ORIGINAL_METADATA_PATH}/shard_{i}.json"

    for data in open(original_metadata_file).readlines():
      data = json.loads(data)
      
      metadata = {
        "id": data["clip_id"],
        "video": f"shard_{i}/{data['clip_id']}.mp4",
        "vision_cap": data["vision_cap"]
      }

      refined_metadata_file = f"{TEMP_REFINED_METADATA_PATH}/shard_{i}.jsonl"
      with open(refined_metadata_file, "a") as f:
        f.write(json.dumps(metadata, separators=(',', ':')) + "\n")

    continue

  # if i != 301:
  #   continue
  
  translated_metadata_file = f"{TRANSLATED_METADATA_PATH}/shard_{i}.json"
  original_metadata_file = f"{ORIGINAL_METADATA_PATH}/shard_{i}.json"

  translated_metadatas = [json.loads(data) for data in open(translated_metadata_file).readlines()]
  translated_ids = [data["id"] for data in translated_metadatas]
  original_metadatas = [json.loads(data) for data in open(original_metadata_file).readlines()]
  original_metadatas = [data for data in original_metadatas if "clip_id" in data and data["clip_id"] in translated_ids ]

  # sort by id
  translated_metadatas.sort(key=lambda x: x["id"])
  original_metadatas.sort(key=lambda x: x["clip_id"])

  assert len(translated_metadatas) == len(original_metadatas), f"{len(translated_metadatas)} != {len(original_metadatas)}"

  # print(translated_metadatas[:10])
  # print(original_metadatas[:10])

  # break

  refined_data = []

  for translated_metadata, original_metadata in tqdm(zip(translated_metadatas, original_metadatas)):
    
    id = translated_metadata["id"]
    clip_id = original_metadata["clip_id"]

    assert id == clip_id, f"{id} != {clip_id}"
    video = translated_metadata["video"]
    vision_cap = original_metadata["vision_cap"]

    refined_data.append({
      "id": id,
      "video": video,
      "vision_cap": vision_cap
    })

  # print(refined_data)

  refined_metadata_file = f"{TEMP_REFINED_METADATA_PATH}/shard_{i}.jsonl"
  with open(refined_metadata_file, "w") as f:
    for data in refined_data:
      f.write(json.dumps(data, separators=(',', ':')) + "\n")
  

100%|██████████| 500/500 [00:00<00:00, 5756.88it/s]
